In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch
import pandas as pd
import numpy as np
import torch.nn as nn
from sklearn.model_selection import train_test_split 
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MinMaxScaler

In [ ]:
#Set the random seed

torch.manual_seed(2022)

In [ ]:
bankrupt = pd.read_csv("/content/Bankruptcy.csv")

bankrupt.head()

,NO,D,YR,R1,R2,R3,R4,R5,R6,R7,...,R15,R16,R17,R18,R19,R20,R21,R22,R23,R24
0,1,0,78,0.23,0.08,0.02,0.03,0.46,0.12,0.19,...,0.05,0.57,0.15,0.23,3.56,0.26,1.55,0.43,0.11,0.17
1,2,0,77,0.19,0.07,0.09,0.12,0.02,0.02,0.03,...,0.09,0.12,0.16,0.22,3.78,1.29,1.40,0.06,0.07,0.10
2,3,0,72,0.07,0.02,0.03,0.05,0.06,0.10,0.14,...,-0.03,0.02,0.02,0.04,13.29,1.61,1.43,0.03,0.05,0.07
3,4,0,80,0.07,0.03,0.04,0.04,0.04,0.06,0.06,...,-0.02,0.01,0.02,0.02,5.36,1.30,1.12,-0.06,-0.08,-0.09
4,5,0,81,0.09,0.02,0.03,0.04,0.06,0.08,0.11,...,0.02,0.07,0.10,0.14,7.74,1.48,1.41,0.03,0.04,0.06


In [ ]:
X = bankrupt.drop(['NO','YR','D'],axis=1).values
y = bankrupt['D'].values
scaler = MinMaxScaler()


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, 
                                                    random_state=42,stratify=y)
X_scl_trn = scaler.fit_transform(X_train) 
X_scl_tst = scaler.transform(X_test) 


In [ ]:
X_torch = torch.from_numpy(X_scl_trn)
y_torch = torch.from_numpy(y_train)
print(X_torch.size())
print(y_torch.size())

torch.Size([92, 24])
torch.Size([92])


In [ ]:
type(X_torch)

torch.Tensor

In [ ]:
# Create a model
model = nn.Sequential(nn.Linear(in_features=X_scl_trn.shape[1], out_features=5),
                      nn.ReLU(),
                      nn.Linear(5, 3),
                      nn.ReLU(),
                      nn.Linear(3,1),
                      nn.Sigmoid())

In [ ]:
criterion = torch.nn.BCELoss()
# Construct the optimizer (Adam in this case)
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)
optimizer

Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: False
    lr: 0.001
    maximize: False
    weight_decay: 0
)

In [ ]:
y_pred = model(X_torch.float())
y_torch = y_torch.unsqueeze(1)
print(y_torch.shape)
print(y_pred.shape)

torch.Size([92, 1])
torch.Size([92, 1])


In [ ]:
# Gradient Descent

for epoch in np.arange(0,1000):
   # Forward pass: Compute predicted y by passing x to the model
   y_pred_prob = model(X_torch.float())

   # Compute and print loss
   loss = criterion(y_pred_prob, y_torch.float())
   print('epoch: ', epoch+1,' loss: ', loss.item())

   # Zero gradients, perform a backward pass, and update the weights.
   optimizer.zero_grad()

   # perform a backward pass (backpropagation)
   loss.backward()

   # Update the parameters
   optimizer.step()
#print('epoch: ', epoch+1,' loss: ', loss.item())

epoch:  1  loss:  0.697766125202179
epoch:  2  loss:  0.6972903609275818
epoch:  3  loss:  0.6968328952789307
epoch:  4  loss:  0.6963897347450256
epoch:  5  loss:  0.6959628462791443
epoch:  6  loss:  0.6955567598342896
epoch:  7  loss:  0.6951721906661987
epoch:  8  loss:  0.6948233842849731
epoch:  9  loss:  0.6944746375083923
epoch:  10  loss:  0.6941453814506531
epoch:  11  loss:  0.6938496828079224
epoch:  12  loss:  0.6935808658599854
epoch:  13  loss:  0.693321168422699
epoch:  14  loss:  0.6930645704269409
epoch:  15  loss:  0.692803144454956
epoch:  16  loss:  0.6925394535064697
epoch:  17  loss:  0.6922755837440491
epoch:  18  loss:  0.6920087933540344
epoch:  19  loss:  0.6917408108711243
epoch:  20  loss:  0.6914815306663513
epoch:  21  loss:  0.6912241578102112
epoch:  22  loss:  0.6909641027450562
epoch:  23  loss:  0.6907011866569519
epoch:  24  loss:  0.6904403567314148
epoch:  25  loss:  0.6901838183403015
epoch:  26  loss:  0.689915657043457
epoch:  27  loss:  0.6896

In [ ]:
X_torch_test = torch.from_numpy(X_scl_tst)
y_pred_prob = model(X_torch_test.float()) # Equivalent predict_proba / predict
y_pred_prob = y_pred_prob.detach().numpy()
y_pred_prob = y_pred_prob.reshape(y_test.shape[0],)
y_pred_prob[:5]

array([0.43275815, 0.9984255 , 0.02015334, 0.99996555, 0.01083817],
      dtype=float32)

In [ ]:
y_pred = np.where(y_pred_prob >= 0.5,1,0)

y_pred[:5]

array([0, 1, 0, 1, 0])

In [ ]:
print(accuracy_score(y_test,y_pred))

0.825


accuracy_score

0.825
